In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
#path= '/content/drive/My Drive/bases_mentoria/primer trimestre 2019'
path= '/content/drive/My Drive/Mentoria/primer trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()


Mounted at /content/drive/


,FECHA,FECHAAPERTURA,TARJETA,CORREDOR,LINEA,SENTIDO
0,01/01/2019 01:32:43 AM,01/01/2019 01:00:11 AM,4992639,Cor 3 Rojo,L35,Ida
1,01/01/2019 02:26:11 AM,01/01/2019 01:00:11 AM,5489022,Cor 3 Rojo,L35,Vuelta
2,01/01/2019 02:26:14 AM,01/01/2019 01:00:11 AM,5489022,Cor 3 Rojo,L35,Vuelta
3,01/01/2019 02:32:27 AM,01/01/2019 01:59:31 AM,5317539,Cor 1 Naranja,L12,Vuelta
4,01/01/2019 02:57:48 AM,01/01/2019 01:59:31 AM,4451141,Cor 1 Naranja,L12,Vuelta


In [ ]:
big_frame['m'] = big_frame.FECHA.astype(str).str[:10].astype(str)

In [ ]:
big_frame['hora'] = big_frame.FECHA.astype(str).str[11:].astype(str)

In [ ]:
big_frame['aux']=big_frame.FECHA.astype(str).str[19:].astype(str) 


In [ ]:
big_frame['aux2']=big_frame.hora.astype(str).str[:2].astype(str) 

In [ ]:
big_frame['hh']=big_frame.aux2 + big_frame.aux

In [ ]:
big_frame.sort_values(by=['hh'], inplace=True)

In [ ]:
big_frame.m.value_counts()

28/03/2019    176887
29/03/2019    176481
13/03/2019    174942
12/03/2019    173855
11/03/2019    173682
               ...  
20/01/2019     34871
27/01/2019     34737
13/01/2019     29166
06/01/2019     22635
01/01/2019     13487
Name: m, Length: 90, dtype: int64

In [ ]:
big_frame['dia'] = pd.to_datetime(big_frame['m'])
big_frame['dia'] = big_frame['dia'].dt.strftime('%d.%m.%Y')

In [ ]:
big_frame.dia.value_counts()

28.03.2019    176887
29.03.2019    176481
13.03.2019    174942
03.12.2019    173855
03.11.2019    173682
               ...  
20.01.2019     34871
27.01.2019     34737
13.01.2019     29166
01.06.2019     22635
01.01.2019     13487
Name: dia, Length: 90, dtype: int64

In [ ]:
big_frame['dia2'] = pd.to_datetime(big_frame['dia'])

In [ ]:

### EJERCICIO 1: INTERACTUAR CON BASES, CONVERTIR FORMATOS FECHAS, GENERAR VARIABLES NUEVAS (MES DIA )###
big_frame['day_of_week'] = big_frame['dia2'].dt.day_name()


In [ ]:
big_frame.dia2.value_counts()

2019-03-28    176887
2019-03-29    176481
2019-03-13    174942
2019-03-12    173855
2019-03-11    173682
               ...  
2019-01-20     34871
2019-01-27     34737
2019-01-13     29166
2019-01-06     22635
2019-01-01     13487
Name: dia2, Length: 90, dtype: int64

In [ ]:
big_frame.shape

(10290050, 14)

In [ ]:
###EJERCICIO 2: 
##APLICAR ESTADÍSTICOS DESCRIPTIVOS SOBRE CORTES DE BOLETOS: 
## 1) VARIABLES ESPACIALES: CORREDOR, LINEA Y SENTIDO
  ## CONTAMOS VIAJES POR  POR CORREDOR, LINEA Y SENTIDO , POR DIA###
big_frame['AUX']=1
big_frame['VIAJES_DIA_CORREDOR']=big_frame.groupby(['CORREDOR', 'dia2'])['AUX'].transform('sum')
big_frame['VIAJES_DIA_LINEA']=big_frame.groupby(['LINEA', 'dia2'])['AUX'].transform('sum')
big_frame['SENTIDO_IDA']=0
mask=big_frame.SENTIDO=="Ida"
big_frame.loc[mask,'SENTIDO_IDA']=1
big_frame['SENTIDO_VUELTA']=0
mask=big_frame.SENTIDO=="Vuelta"
big_frame.loc[mask,'SENTIDO_VUELTA']=1

big_frame['VIAJES_DIA_LINEA_SENT_IDA']=big_frame.groupby(['LINEA', 'dia2'])['SENTIDO_IDA'].transform('sum')
big_frame['VIAJES_DIA_LINEA_SENT_VUELTA']=big_frame.groupby(['LINEA', 'dia2'])['SENTIDO_VUELTA'].transform('sum')
big_frame.head(10)

big_frame= big_frame.drop_duplicates(['CORREDOR', 'LINEA', 'dia2', 'VIAJES_DIA_CORREDOR', 	'VIAJES_DIA_LINEA' , 'VIAJES_DIA_LINEA_SENT_IDA', 	'VIAJES_DIA_LINEA_SENT_VUELTA' ])
big_frame.head()
big_frame=big_frame.loc[:,['CORREDOR', 'LINEA', 'dia2',  'VIAJES_DIA_CORREDOR', 	'VIAJES_DIA_LINEA' , 'VIAJES_DIA_LINEA_SENT_IDA', 	'VIAJES_DIA_LINEA_SENT_VUELTA', 'day_of_week']]
#df1.to_csv("df1.csv", index=None)
#from google.colab import files
#files.download('df1.csv') 

## 2) VARIABLES TEMPORALES: MES DIA DE LA SEMANA Y HORA##
## 3) ANALIZAR ESTACIONALIDAD
## IDEM SOBRE TARJETAS MESES USADAS VS DIAS PROMEDIO USADAS (¿OUTLIERS?, CANTIDAD DE USOS DIA)

###LUEGO GUARDAMOS UN DF PARA PODER LUEGO HACER LOOP, DIGAMOS, DEBERIA HABER 2 LOOP, EL QUE LEVANTA Y PROCESA Y LUEGO EL QEU UNE TODO##

KeyError: ignored

In [ ]:
big_frame.shape

(6210, 9)

In [ ]:
big_frame.dia2.value_counts()

2019-02-06    72
2019-01-21    72
2019-01-22    72
2019-01-23    72
2019-03-18    72
              ..
2019-01-06    66
2019-02-16    66
2019-03-10    66
2019-03-24    66
2019-01-01    66
Name: dia2, Length: 90, dtype: int64

In [ ]:
viajes_1t19= big_frame
viajes_1t19.to_csv("viajes_1t19.csv", index=None)
from google.colab import files
files.download('viajes_1t19.csv') 



In [ ]:
##LO MISMO PARA EL SEGUNDO TRIMESTRE DE 2019#
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
path= '/content/drive/My Drive/Mentoria/segundo trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()




In [ ]:
big_frame['m'] = big_frame.FECHA.astype(str).str[:10].astype(str)

In [ ]:
big_frame.m.value_counts()

In [ ]:
big_frame['dia'] = pd.to_datetime(big_frame['m'])
big_frame['dia'] = big_frame['dia'].dt.strftime('%d.%m.%Y')

In [ ]:
big_frame.dia.value_counts()

In [ ]:
big_frame['dia2'] = pd.to_datetime(big_frame['dia'])

In [ ]:
### EJERCICIO 1: INTERACTUAR CON BASES, CONVERTIR FORMATOS FECHAS, GENERAR VARIABLES NUEVAS (MES DIA )###
big_frame['day_of_week'] = big_frame['dia2'].dt.day_name()
big_frame['FECHA2'] = pd.to_datetime(big_frame['FECHA'])
big_frame['hora'] = big_frame['FECHA2'].dt.hour
big_frame.head()

In [ ]:
big_frame.dia2.value_counts()

In [ ]:
big_frame.shape

In [ ]:
###EJERCICIO 2: 
##APLICAR ESTADÍSTICOS DESCRIPTIVOS SOBRE CORTES DE BOLETOS: 
## 1) VARIABLES ESPACIALES: CORREDOR, LINEA Y SENTIDO
  ## CONTAMOS VIAJES POR  POR CORREDOR, LINEA Y SENTIDO , POR DIA###
big_frame['AUX']=1
big_frame['VIAJES_DIA_CORREDOR']=big_frame.groupby(['CORREDOR', 'dia2'])['AUX'].transform('sum')
big_frame['VIAJES_DIA_LINEA']=big_frame.groupby(['LINEA', 'dia2'])['AUX'].transform('sum')
big_frame['SENTIDO_IDA']=0
mask=big_frame.SENTIDO=="Ida"
big_frame.loc[mask,'SENTIDO_IDA']=1
big_frame['SENTIDO_VUELTA']=0
mask=big_frame.SENTIDO=="Vuelta"
big_frame.loc[mask,'SENTIDO_VUELTA']=1

big_frame['VIAJES_DIA_LINEA_SENT_IDA']=big_frame.groupby(['LINEA', 'dia2'])['SENTIDO_IDA'].transform('sum')
big_frame['VIAJES_DIA_LINEA_SENT_VUELTA']=big_frame.groupby(['LINEA', 'dia2'])['SENTIDO_VUELTA'].transform('sum')
big_frame.head(10)

big_frame= big_frame.drop_duplicates(['CORREDOR', 'LINEA', 'dia2', 'VIAJES_DIA_CORREDOR', 	'VIAJES_DIA_LINEA' , 'VIAJES_DIA_LINEA_SENT_IDA', 	'VIAJES_DIA_LINEA_SENT_VUELTA' ])
big_frame.head()
big_frame=big_frame.loc[:,['CORREDOR', 'LINEA', 'dia2',  'VIAJES_DIA_CORREDOR', 	'VIAJES_DIA_LINEA' , 'VIAJES_DIA_LINEA_SENT_IDA', 	'VIAJES_DIA_LINEA_SENT_VUELTA', 'hora', 'day_of_week']]
#df1.to_csv("df1.csv", index=None)
#from google.colab import files
#files.download('df1.csv') 

## 2) VARIABLES TEMPORALES: MES DIA DE LA SEMANA Y HORA##
## 3) ANALIZAR ESTACIONALIDAD
## IDEM SOBRE TARJETAS MESES USADAS VS DIAS PROMEDIO USADAS (¿OUTLIERS?, CANTIDAD DE USOS DIA)

###LUEGO GUARDAMOS UN DF PARA PODER LUEGO HACER LOOP, DIGAMOS, DEBERIA HABER 2 LOOP, EL QUE LEVANTA Y PROCESA Y LUEGO EL QEU UNE TODO##

In [ ]:
big_frame.shape

In [ ]:
big_frame.dia2.value_counts()

In [ ]:
viajes_2t19= big_frame
viajes_2t19.to_csv("viajes_2t19.csv", index=None)
from google.colab import files
files.download('viajes_2t19.csv') 


In [ ]:
##LO MISMO PARA EL TERCER TRIMESTRE DE 2019#
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
path= '/content/drive/My Drive/Mentoria/tercer trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()




In [ ]:
big_frame['m'] = big_frame.FECHA.astype(str).str[:10].astype(str)

In [ ]:
big_frame.m.value_counts()

In [ ]:
big_frame['dia'] = pd.to_datetime(big_frame['m'])
big_frame['dia'] = big_frame['dia'].dt.strftime('%d.%m.%Y')

In [ ]:
big_frame.dia.value_counts()

In [ ]:
big_frame['dia2'] = pd.to_datetime(big_frame['dia'])

In [ ]:
### EJERCICIO 1: INTERACTUAR CON BASES, CONVERTIR FORMATOS FECHAS, GENERAR VARIABLES NUEVAS (MES DIA )###
big_frame['day_of_week'] = big_frame['dia2'].dt.day_name()
big_frame['FECHA2'] = pd.to_datetime(big_frame['FECHA'])
big_frame['hora'] = big_frame['FECHA2'].dt.hour
big_frame.head()

In [ ]:
big_frame.dia2.value_counts()

In [ ]:
big_frame.shape

In [ ]:
###EJERCICIO 2: 
##APLICAR ESTADÍSTICOS DESCRIPTIVOS SOBRE CORTES DE BOLETOS: 
## 1) VARIABLES ESPACIALES: CORREDOR, LINEA Y SENTIDO
  ## CONTAMOS VIAJES POR  POR CORREDOR, LINEA Y SENTIDO , POR DIA###
big_frame['AUX']=1
big_frame['VIAJES_DIA_CORREDOR']=big_frame.groupby(['CORREDOR', 'dia2'])['AUX'].transform('sum')
big_frame['VIAJES_DIA_LINEA']=big_frame.groupby(['LINEA', 'dia2'])['AUX'].transform('sum')
big_frame['SENTIDO_IDA']=0
mask=big_frame.SENTIDO=="Ida"
big_frame.loc[mask,'SENTIDO_IDA']=1
big_frame['SENTIDO_VUELTA']=0
mask=big_frame.SENTIDO=="Vuelta"
big_frame.loc[mask,'SENTIDO_VUELTA']=1

big_frame['VIAJES_DIA_LINEA_SENT_IDA']=big_frame.groupby(['LINEA', 'dia2'])['SENTIDO_IDA'].transform('sum')
big_frame['VIAJES_DIA_LINEA_SENT_VUELTA']=big_frame.groupby(['LINEA', 'dia2'])['SENTIDO_VUELTA'].transform('sum')
big_frame.head(10)

big_frame= big_frame.drop_duplicates(['CORREDOR', 'LINEA', 'dia2', 'VIAJES_DIA_CORREDOR', 	'VIAJES_DIA_LINEA' , 'VIAJES_DIA_LINEA_SENT_IDA', 	'VIAJES_DIA_LINEA_SENT_VUELTA' ])
big_frame.head()
big_frame=big_frame.loc[:,['CORREDOR', 'LINEA', 'dia2',  'VIAJES_DIA_CORREDOR', 	'VIAJES_DIA_LINEA' , 'VIAJES_DIA_LINEA_SENT_IDA', 	'VIAJES_DIA_LINEA_SENT_VUELTA', 'hora', 'day_of_week']]
#df1.to_csv("df1.csv", index=None)
#from google.colab import files
#files.download('df1.csv') 

## 2) VARIABLES TEMPORALES: MES DIA DE LA SEMANA Y HORA##
## 3) ANALIZAR ESTACIONALIDAD
## IDEM SOBRE TARJETAS MESES USADAS VS DIAS PROMEDIO USADAS (¿OUTLIERS?, CANTIDAD DE USOS DIA)

###LUEGO GUARDAMOS UN DF PARA PODER LUEGO HACER LOOP, DIGAMOS, DEBERIA HABER 2 LOOP, EL QUE LEVANTA Y PROCESA Y LUEGO EL QEU UNE TODO##

In [ ]:
big_frame.shape

In [ ]:
big_frame.dia2.value_counts()

In [ ]:
viajes_3t19= big_frame
viajes_3t19.to_csv("viajes_3t19.csv", index=None)
from google.colab import files
files.download('viajes_3t19.csv') 

In [ ]:
##LO MISMO PARA EL CUARTO TRIMESTRE DE 2019#
from google.colab import drive
drive.mount('/content/drive/', force_remount=True )
import pandas as pd
#%cd /content/drive/My Drive/bases_mentoria/
path= '/content/drive/My Drive/Mentoria/cuarto trimestre 2019'
import glob
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, encoding='latin-1', sep=";"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

big_frame.head()

In [ ]:
big_frame['m'] = big_frame.FECHA.astype(str).str[:10].astype(str)

In [ ]:
big_frame.m.value_counts()

In [ ]:
big_frame['dia'] = pd.to_datetime(big_frame['m'])
big_frame['dia'] = big_frame['dia'].dt.strftime('%d.%m.%Y')

In [ ]:
big_frame.info()

In [ ]:
big_frame.dia.value_counts()

In [ ]:
big_frame['dia2'] = pd.to_datetime(big_frame['dia'])

In [ ]:
### EJERCICIO 1: INTERACTUAR CON BASES, CONVERTIR FORMATOS FECHAS, GENERAR VARIABLES NUEVAS (MES DIA )###
big_frame['day_of_week'] = big_frame['dia2'].dt.day_name()
big_frame['FECHA2'] = pd.to_datetime(big_frame['FECHA'])
big_frame['hora'] = big_frame['FECHA2'].dt.hour
big_frame.head()

In [ ]:
big_frame.dia2.value_counts()

In [ ]:
big_frame.shape

In [ ]:
###EJERCICIO 2: 
##APLICAR ESTADÍSTICOS DESCRIPTIVOS SOBRE CORTES DE BOLETOS: 
## 1) VARIABLES ESPACIALES: CORREDOR, LINEA Y SENTIDO
  ## CONTAMOS VIAJES POR  POR CORREDOR, LINEA Y SENTIDO , POR DIA###
big_frame['AUX']=1
big_frame['VIAJES_DIA_CORREDOR']=big_frame.groupby(['CORREDOR', 'dia2'])['AUX'].transform('sum')
big_frame['VIAJES_DIA_LINEA']=big_frame.groupby(['LINEA', 'dia2'])['AUX'].transform('sum')
big_frame['SENTIDO_IDA']=0
mask=big_frame.SENTIDO=="Ida"
big_frame.loc[mask,'SENTIDO_IDA']=1
big_frame['SENTIDO_VUELTA']=0
mask=big_frame.SENTIDO=="Vuelta"
big_frame.loc[mask,'SENTIDO_VUELTA']=1

big_frame['VIAJES_DIA_LINEA_SENT_IDA']=big_frame.groupby(['LINEA', 'dia2'])['SENTIDO_IDA'].transform('sum')
big_frame['VIAJES_DIA_LINEA_SENT_VUELTA']=big_frame.groupby(['LINEA', 'dia2'])['SENTIDO_VUELTA'].transform('sum')
big_frame.head(10)

big_frame= big_frame.drop_duplicates(['CORREDOR', 'LINEA', 'dia2', 'VIAJES_DIA_CORREDOR', 	'VIAJES_DIA_LINEA' , 'VIAJES_DIA_LINEA_SENT_IDA', 	'VIAJES_DIA_LINEA_SENT_VUELTA' ])
big_frame.head()
big_frame=big_frame.loc[:,['CORREDOR', 'LINEA', 'dia2',  'VIAJES_DIA_CORREDOR', 	'VIAJES_DIA_LINEA' , 'VIAJES_DIA_LINEA_SENT_IDA', 	'VIAJES_DIA_LINEA_SENT_VUELTA', 'hora', 'day_of_week']]
#df1.to_csv("df1.csv", index=None)
#from google.colab import files
#files.download('df1.csv') 

## 2) VARIABLES TEMPORALES: MES DIA DE LA SEMANA Y HORA##
## 3) ANALIZAR ESTACIONALIDAD
## IDEM SOBRE TARJETAS MESES USADAS VS DIAS PROMEDIO USADAS (¿OUTLIERS?, CANTIDAD DE USOS DIA)

###LUEGO GUARDAMOS UN DF PARA PODER LUEGO HACER LOOP, DIGAMOS, DEBERIA HABER 2 LOOP, EL QUE LEVANTA Y PROCESA Y LUEGO EL QEU UNE TODO##

In [ ]:
big_frame.shape

In [ ]:
big_frame.dia2.value_counts()

In [ ]:
viajes_4t19= big_frame
viajes_4t19.to_csv("viajes_4t19.csv", index=None)
from google.colab import files
files.download('viajes_4t19.csv') 